# Difference between Aqua and Terra
Looking at the classifications of the specific pattern a clear dependence on the satellite is visible. This notebook shall illuminate, whether these differences are real.

In [1]:
import csv
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd

import numpy as np
import seaborn as sns
import datetime as datetime
from netCDF4 import num2date, date2num
import zooniverse as zn

from PIL import Image
import matplotlib.patches as patches
import matplotlib.image as mpimage

%load_ext autoreload
%autoreload 2

In [3]:
classfile_in = '../zooniverse_raw/sugar-flower-fish-or-gravel-classifications_18_11_30.csv'
subject_in = '../zooniverse_raw/sugar-flower-fish-or-gravel-subjects_18_11_05.csv'
image_path = '/Volumes/Samsung_T5/Data/Satellites/MODIS/{}/'

In [4]:
classification_data = zn.parse_classifications(classfile_in,json_columns=['metadata', 'annotations', 'subject_data']); classification_data.tail()

,classification_id,user_name,user_id,user_ip,workflow_id,workflow_name,workflow_version,created_at,gold_standard,expert,metadata,annotations,subject_data,subject_ids
30056,133973344,szantai,1848018.0,2076dfcfe93e5cbf253d,8072,Practice,24.13,2018-11-29 22:40:44 UTC,NaN,NaN,"{'source': 'api', 'session': 'aef00130a923bbef...","{'task': 'T0', 'task_label': 'Draw bounding bo...","{'27140160': {'retired': None, 'fn': '/project...",27140160
30057,133973553,szantai,1848018.0,2076dfcfe93e5cbf253d,8072,Practice,24.13,2018-11-29 22:42:32 UTC,NaN,NaN,"{'source': 'api', 'session': 'aef00130a923bbef...","{'task': 'T0', 'task_label': 'Draw bounding bo...","{'27140298': {'retired': None, 'fn': '/project...",27140298
30058,133973786,szantai,1848018.0,2076dfcfe93e5cbf253d,8072,Practice,24.13,2018-11-29 22:45:19 UTC,NaN,NaN,"{'source': 'api', 'session': 'aef00130a923bbef...","{'task': 'T0', 'task_label': 'Draw bounding bo...","{'27140428': {'retired': None, 'fn': '/project...",27140428
30059,133974061,szantai,1848018.0,2076dfcfe93e5cbf253d,8072,Practice,24.13,2018-11-29 22:47:26 UTC,NaN,NaN,"{'source': 'api', 'session': 'aef00130a923bbef...","{'task': 'T0', 'task_label': 'Draw bounding bo...","{'27140219': {'retired': None, 'fn': '/project...",27140219
30060,133974208,szantai,1848018.0,2076dfcfe93e5cbf253d,8072,Practice,24.13,2018-11-29 22:48:45 UTC,NaN,NaN,"{'source': 'api', 'session': 'aef00130a923bbef...","{'task': 'T0', 'task_label': 'Draw bounding bo...","{'27140510': {'retired': None, 'fn': '/project...",27140510


In [5]:
subject_data = zn.load_classifications(subject_in)

In [6]:
def retrieve_fn(x):
    try:
        return x["fn"]
    except:
        return None

In [7]:
subject_data["filename"] = subject_data["metadata"].apply(retrieve_fn)

## Select full dataset workflow (8073) only

In [8]:
fulldataset = classification_data[classification_data.workflow_id == 8073];

In [9]:
fulldataset.subject_data.iloc[0]

{'26976410': {'fn': '/project/meteo/work/S.Rasp/cloud-classification/Region1_DJF_Aqua/Aqua_CorrectedReflectance20070103_-61--40_10-24.jpeg',
  'id': '2',
  'retired': None}}

## Find subject/image pairs of terra and aqua satellites

In [10]:
def devide_dataset(x):
    if x is None: return None
    elif "Aqua" in x: return int(0)
    elif "Terra" in x: return int(1)

In [11]:
subject_data["satellite"] = subject_data["filename"].apply(devide_dataset)

In [12]:
[[_, aqua_grp], [_, terra_grp]]= subject_data.groupby('satellite')

In [13]:
subject_pairs = []
for i, subject in aqua_grp.iterrows():
    id_aqua = subject.subject_id
    fn_terra = subject.filename.replace("Aqua","Terra")
    ind_terra = np.where([fn == fn_terra for fn in terra_grp.filename.values.astype(str)])[0]
    if np.size(ind_terra) > 0:
        id_terra = terra_grp.iloc[np.int(ind_terra)].subject_id
        subject_pairs.append([id_aqua, id_terra])

In [14]:
subject_data.set_index('subject_id',inplace=True)

In [15]:
def extract_labels(annotations):
    annos = []
    for a in annotations:
        for box in a['value']:
            annos.append([box['x'], box['y'], box['width'],
                          box['height'], box['tool_label']
            ])
    return annos

In [ ]:
pattern = 'Flower'
for p,(terra_id, aqua_id) in enumerate(subject_pairs[900:]):
    if terra_id is not None and aqua_id is not None:
        aqua_pattern = False
        terra_pattern = False
        terra_classification = fulldataset[fulldataset.subject_ids == terra_id]
        aqua_classification  = fulldataset[fulldataset.subject_ids == aqua_id]
        boxes_aqua  = extract_labels(aqua_classification.annotations.values)
        boxes_terra = extract_labels(terra_classification.annotations.values)
        for box in boxes_aqua:
            if box[4] == pattern: aqua_pattern = True
        for box in boxes_terra:
            if box[4] == pattern: terra_pattern = True
        if aqua_pattern==False and terra_pattern==True:
            # Sugar was in aqua but not in terra overpass
            try:
                fig, ax = plt.subplots(1,2,figsize=(15, 15),dpi=70)
                terra_filename = subject_data.loc[terra_id].filename
                if type(terra_filename) != type('r'): terra_filename = subject_data.loc[terra_id].filename.values[0]
                img_VIS = Image.open(image_path.format('VIS_images')+"/".join(terra_filename.split('/')[-2:]))
                ax[0].imshow(img_VIS)
                for a in boxes_terra:
                    rect = patches.Rectangle((a[0],a[1]), a[2], a[3], facecolor='none', edgecolor='r')
                    ax[0].add_patch(rect)
                    ax[0].text(a[0],a[1], a[4], color='red', fontsize=15, va='top')
                ax[0].axis("off");

                aqua_filename = subject_data.loc[aqua_id].filename
                if type(aqua_filename) != type('r'): aqua_filename = subject_data.loc[aqua_id].filename.values[0]
                img_VIS2 = Image.open(image_path.format('VIS_images')+"/".join(aqua_filename.split('/')[-2:]))
                plt.imshow(img_VIS2)
                for a in boxes_aqua:
                    rect2 = patches.Rectangle((a[0],a[1]), a[2], a[3], facecolor='none', edgecolor='r')
                    ax[1].add_patch(rect2)
                    ax[1].text(a[0],a[1], a[4], color='red', fontsize=15, va='top')
                ax[1].axis("off");
                plt.show();
                plt.close(fig);
                print(p)
            except TypeError:
                continue

In [1]:
subject_data.loc[terra_id]

NameError: name 'subject_data' is not defined

'Region3_SON_Terra/Terra_CorrectedReflectance20171130_-135--114_-15--1.jpeg'

In [174]:
subject_data.loc[terra_id].filename.values

array(['/project/meteo/work/S.Rasp/cloud-classification/Region1_DJF_Aqua/Aqua_CorrectedReflectance20070213_-61--40_10-24.jpeg',
       '/project/meteo/work/S.Rasp/cloud-classification/Region1_DJF_Aqua/Aqua_CorrectedReflectance20070213_-61--40_10-24.jpeg'],
      dtype=object)

In [ ]:
a[]